# Import dependencies and cleaning data

In [1]:
!pip install -U imbalanced-learn

Requirement already up-to-date: imbalanced-learn in c:\users\kirst\appdata\local\continuum\anaconda3\lib\site-packages (0.7.0)


In [2]:
import pandas as pd
import os
from sklearn import tree
from sklearn.model_selection import GridSearchCV

In [3]:
data = pd.read_csv('germanHeaders.csv', header=None)

In [4]:
columns = data.iloc[0].tolist()

In [5]:
data.columns = columns

In [6]:
data = data.iloc[1:]

In [7]:
data = data.iloc[:,1:]

In [8]:
data.columns.tolist()

['account_status',
 'duration',
 'credit_history',
 'purpose',
 'credit_amount',
 'savings_account',
 'unemployed',
 'installment_rate',
 'personal_status_sex',
 'debtors/guarantors',
 'present_residence_since',
 'property',
 'age',
 'other_installment_plans',
 'housing',
 'credits',
 'job',
 'liable_for',
 'telephone',
 'foreign_worker',
 'good/bad']

In [9]:
data

,account_status,duration,credit_history,purpose,credit_amount,savings_account,unemployed,installment_rate,personal_status_sex,debtors/guarantors,...,property,age,other_installment_plans,housing,credits,job,liable_for,telephone,foreign_worker,good/bad
1,A11,6,A34,A43,1169,A65,A75,4,A93,A101,...,A121,67,A143,A152,2,A173,1,A192,A201,1
2,A12,48,A32,A43,5951,A61,A73,2,A92,A101,...,A121,22,A143,A152,1,A173,1,A191,A201,2
3,A14,12,A34,A46,2096,A61,A74,2,A93,A101,...,A121,49,A143,A152,1,A172,2,A191,A201,1
4,A11,42,A32,A42,7882,A61,A74,2,A93,A103,...,A122,45,A143,A153,1,A173,2,A191,A201,1
5,A11,24,A33,A40,4870,A61,A73,3,A93,A101,...,A124,53,A143,A153,2,A173,2,A191,A201,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,A14,12,A32,A42,1736,A61,A74,3,A92,A101,...,A121,31,A143,A152,1,A172,1,A191,A201,1
997,A11,30,A32,A41,3857,A61,A73,4,A91,A101,...,A122,40,A143,A152,1,A174,1,A192,A201,1
998,A14,12,A32,A43,804,A61,A75,4,A93,A101,...,A123,38,A143,A152,1,A173,1,A191,A201,1
999,A11,45,A32,A43,1845,A61,A73,4,A93,A101,...,A124,23,A143,A153,1,A173,1,A192,A201,2


In [10]:
a_columns = ['account_status', 'credit_history', 'purpose', 'savings_account', 'unemployed',
     'personal_status_sex', 'debtors/guarantors', 'property', 'other_installment_plans', 'housing', 'job', 'telephone', 'foreign_worker']

In [11]:
data[a_columns] = data[a_columns].replace({'A':''}, regex=True)
data['good/bad'] = data['good/bad'].replace({'2':'0'}, regex=True)

In [12]:
data[a_columns] = data[a_columns].apply(pd.to_numeric) 
data

,account_status,duration,credit_history,purpose,credit_amount,savings_account,unemployed,installment_rate,personal_status_sex,debtors/guarantors,...,property,age,other_installment_plans,housing,credits,job,liable_for,telephone,foreign_worker,good/bad
1,11,6,34,43,1169,65,75,4,93,101,...,121,67,143,152,2,173,1,192,201,1
2,12,48,32,43,5951,61,73,2,92,101,...,121,22,143,152,1,173,1,191,201,0
3,14,12,34,46,2096,61,74,2,93,101,...,121,49,143,152,1,172,2,191,201,1
4,11,42,32,42,7882,61,74,2,93,103,...,122,45,143,153,1,173,2,191,201,1
5,11,24,33,40,4870,61,73,3,93,101,...,124,53,143,153,2,173,2,191,201,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,14,12,32,42,1736,61,74,3,92,101,...,121,31,143,152,1,172,1,191,201,1
997,11,30,32,41,3857,61,73,4,91,101,...,122,40,143,152,1,174,1,192,201,1
998,14,12,32,43,804,61,75,4,93,101,...,123,38,143,152,1,173,1,191,201,1
999,11,45,32,43,1845,61,73,4,93,101,...,124,23,143,153,1,173,1,192,201,0


In [13]:
#data.to_csv('cleandata_woutcome.csv')

# Assign variables and adjust sampling

In [14]:
X = data.drop("good/bad", axis=1)
y = data["good/bad"]
attribute_names = X.columns
#X.head()

In [15]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X, y)

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, random_state=1)

In [17]:
y_train.value_counts()

0    527
1    523
Name: good/bad, dtype: int64

# Decision Tree

In [18]:
dtc = tree.DecisionTreeClassifier(random_state=1)
dtc.fit(X_train, y_train)

#y_pred_class = dtc.predict(X_test)
dtc.score(X_test, y_test)

0.8228571428571428

In [19]:
# test different parameters
# try gini / entropy criteria

sample_split_range = list(range(1,10))
param_grid = dict(min_samples_split=sample_split_range)

grid = GridSearchCV(dtc, param_grid, verbose=3)
grid.fit(X_train, y_train)

Fitting 5 folds for each of 199 candidates, totalling 995 fits
[CV] min_samples_split=1 .............................................
[CV] ................... min_samples_split=1, score=nan, total=   0.0s
[CV] min_samples_split=1 .............................................
[CV] ................... min_samples_split=1, score=nan, total=   0.0s
[CV] min_samples_split=1 .............................................
[CV] ................... min_samples_split=1, score=nan, total=   0.0s
[CV] min_samples_split=1 .............................................
[CV] ................... min_samples_split=1, score=nan, total=   0.0s
[CV] min_samples_split=1 .............................................
[CV] ................... min_samples_split=1, score=nan, total=   0.0s
[CV] min_samples_split=2 .............................................
[CV] ................. min_samples_split=2, score=0.795, total=   0.0s
[CV] min_samples_split=2 .............................................
[CV] .........

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\kirst\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\kirst\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\kirst\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 894, in fit
    X_idx_sorted=X_idx_sorted)
  File "C:\Users\kirst\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 231, in fit
    % self.min_samples_split)
ValueError: min_samples_split must be an integer greater than 1 or a float in (0.0, 1.0]; got the integer 1

  FitFailedWarning)
[Parallel(n_jobs=1)]:

[CV] ................. min_samples_split=5, score=0.724, total=   0.0s
[CV] min_samples_split=5 .............................................
[CV] ................. min_samples_split=5, score=0.776, total=   0.0s
[CV] min_samples_split=5 .............................................
[CV] ................. min_samples_split=5, score=0.757, total=   0.0s
[CV] min_samples_split=5 .............................................
[CV] ................. min_samples_split=5, score=0.795, total=   0.0s
[CV] min_samples_split=5 .............................................
[CV] ................. min_samples_split=5, score=0.786, total=   0.0s
[CV] min_samples_split=6 .............................................
[CV] ................. min_samples_split=6, score=0.748, total=   0.0s
[CV] min_samples_split=6 .............................................
[CV] ................. min_samples_split=6, score=0.776, total=   0.0s
[CV] min_samples_split=6 .............................................
[CV] .

[CV] ................ min_samples_split=17, score=0.752, total=   0.0s
[CV] min_samples_split=17 ............................................
[CV] ................ min_samples_split=17, score=0.757, total=   0.0s
[CV] min_samples_split=17 ............................................
[CV] ................ min_samples_split=17, score=0.700, total=   0.0s
[CV] min_samples_split=17 ............................................
[CV] ................ min_samples_split=17, score=0.729, total=   0.0s
[CV] min_samples_split=18 ............................................
[CV] ................ min_samples_split=18, score=0.700, total=   0.0s
[CV] min_samples_split=18 ............................................
[CV] ................ min_samples_split=18, score=0.743, total=   0.0s
[CV] min_samples_split=18 ............................................
[CV] ................ min_samples_split=18, score=0.748, total=   0.0s
[CV] min_samples_split=18 ............................................
[CV] .

[CV] ................ min_samples_split=29, score=0.719, total=   0.0s
[CV] min_samples_split=29 ............................................
[CV] ................ min_samples_split=29, score=0.710, total=   0.0s
[CV] min_samples_split=30 ............................................
[CV] ................ min_samples_split=30, score=0.719, total=   0.0s
[CV] min_samples_split=30 ............................................
[CV] ................ min_samples_split=30, score=0.752, total=   0.0s
[CV] min_samples_split=30 ............................................
[CV] ................ min_samples_split=30, score=0.752, total=   0.0s
[CV] min_samples_split=30 ............................................
[CV] ................ min_samples_split=30, score=0.724, total=   0.0s
[CV] min_samples_split=30 ............................................
[CV] ................ min_samples_split=30, score=0.710, total=   0.0s
[CV] min_samples_split=31 ............................................
[CV] .

...

[CV] ............... min_samples_split=172, score=0.676, total=   0.0s
[CV] min_samples_split=172 ...........................................
[CV] ............... min_samples_split=172, score=0.686, total=   0.0s
[CV] min_samples_split=172 ...........................................
[CV] ............... min_samples_split=172, score=0.719, total=   0.0s
[CV] min_samples_split=172 ...........................................
[CV] ............... min_samples_split=172, score=0.733, total=   0.0s
[CV] min_samples_split=173 ...........................................
[CV] ............... min_samples_split=173, score=0.686, total=   0.0s
[CV] min_samples_split=173 ...........................................
[CV] ............... min_samples_split=173, score=0.676, total=   0.0s
[CV] min_samples_split=173 ...........................................
[CV] ............... min_samples_split=173, score=0.686, total=   0.0s
[CV] min_samples_split=173 ...........................................
[CV] .

[CV] ............... min_samples_split=185, score=0.686, total=   0.0s
[CV] min_samples_split=185 ...........................................
[CV] ............... min_samples_split=185, score=0.724, total=   0.0s
[CV] min_samples_split=185 ...........................................
[CV] ............... min_samples_split=185, score=0.738, total=   0.0s
[CV] min_samples_split=186 ...........................................
[CV] ............... min_samples_split=186, score=0.686, total=   0.0s
[CV] min_samples_split=186 ...........................................
[CV] ............... min_samples_split=186, score=0.643, total=   0.0s
[CV] min_samples_split=186 ...........................................
[CV] ............... min_samples_split=186, score=0.686, total=   0.0s
[CV] min_samples_split=186 ...........................................
[CV] ............... min_samples_split=186, score=0.724, total=   0.0s
[CV] min_samples_split=186 ...........................................
[CV] .

[CV] ............... min_samples_split=198, score=0.719, total=   0.0s
[CV] min_samples_split=199 ...........................................
[CV] ............... min_samples_split=199, score=0.686, total=   0.0s
[CV] min_samples_split=199 ...........................................
[CV] ............... min_samples_split=199, score=0.643, total=   0.0s
[CV] min_samples_split=199 ...........................................
[CV] ............... min_samples_split=199, score=0.686, total=   0.0s
[CV] min_samples_split=199 ...........................................
[CV] ............... min_samples_split=199, score=0.724, total=   0.0s
[CV] min_samples_split=199 ...........................................
[CV] ............... min_samples_split=199, score=0.719, total=   0.0s


[Parallel(n_jobs=1)]: Done 995 out of 995 | elapsed:    8.8s finished


GridSearchCV(estimator=DecisionTreeClassifier(random_state=1),
             param_grid={'min_samples_split': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
                                               11, 12, 13, 14, 15, 16, 17, 18,
                                               19, 20, 21, 22, 23, 24, 25, 26,
                                               27, 28, 29, 30, ...]},
             verbose=3)

In [20]:
print(grid.best_score_)
print(grid.best_params_)

0.800952380952381
{'min_samples_split': 2}


In [21]:
predictions = grid.predict(X_test)

In [22]:
from sklearn.metrics import classification_report
print (classification_report(y_test, predictions,
                            target_names=["bad","good"]))

              precision    recall  f1-score   support

         bad       0.77      0.91      0.84       173
        good       0.89      0.74      0.81       177

    accuracy                           0.82       350
   macro avg       0.83      0.82      0.82       350
weighted avg       0.83      0.82      0.82       350



# Random Forest

In [23]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=2000)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.8885714285714286

In [24]:
sorted(zip(rf.feature_importances_, attribute_names),reverse=True)

[(0.14920985662926212, 'account_status'),
 (0.12379129870138347, 'credit_amount'),
 (0.0994441046726705, 'age'),
 (0.08988715526184872, 'duration'),
 (0.06349179438899528, 'credit_history'),
 (0.057141059208436214, 'purpose'),
 (0.05378071392696873, 'savings_account'),
 (0.05225438783269891, 'unemployed'),
 (0.04420777036124486, 'property'),
 (0.041449687847142674, 'installment_rate'),
 (0.038239875096373006, 'present_residence_since'),
 (0.031333205262760636, 'job'),
 (0.030579156151204906, 'personal_status_sex'),
 (0.02565780797810622, 'other_installment_plans'),
 (0.023140180661672168, 'housing'),
 (0.022108011941725146, 'debtors/guarantors'),
 (0.02036315570562195, 'credits'),
 (0.017690871127187507, 'telephone'),
 (0.011628509155627381, 'liable_for'),
 (0.00460139808906966, 'foreign_worker')]